In [1]:
import highway_env
import stable_baselines3
import torch

In [2]:
name="highspeed-rew-1"

In [3]:
model = stable_baselines3.DQN.load(f"{name}.zip")

/home/samuel/anaconda3/envs/abz/lib/python3.10/site-packages/stable_baselines3/common/save_util.py:437: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  th_object = th.load(fil

In [4]:
model.policy

DQNPolicy(
  (q_net): QNetwork(
    (features_extractor): FlattenExtractor(
      (flatten): Flatten(start_dim=1, end_dim=-1)
    )
    (q_net): Sequential(
      (0): Linear(in_features=25, out_features=256, bias=True)
      (1): ReLU()
      (2): Linear(in_features=256, out_features=256, bias=True)
      (3): ReLU()
      (4): Linear(in_features=256, out_features=3, bias=True)
    )
  )
  (q_net_target): QNetwork(
    (features_extractor): FlattenExtractor(
      (flatten): Flatten(start_dim=1, end_dim=-1)
    )
    (q_net): Sequential(
      (0): Linear(in_features=25, out_features=256, bias=True)
      (1): ReLU()
      (2): Linear(in_features=256, out_features=256, bias=True)
      (3): ReLU()
      (4): Linear(in_features=256, out_features=3, bias=True)
    )
  )
)

In [5]:
class OnnxableSB3Policy(torch.nn.Module):
    def __init__(self, policy):
        super().__init__()
        self.policy = policy

    def forward(self, observation):
        # NOTE: Preprocessing is included, but postprocessing
        # (clipping/inscaling actions) is not,
        # If needed, you also need to transpose the images so that they are channel first
        # use deterministic=False if you want to export the stochastic policy
        # policy() returns `actions, values, log_prob` for PPO
        return self.policy(observation, deterministic=True)

In [6]:
onnx_policy = OnnxableSB3Policy(model.policy)

In [7]:
observation_size = model.observation_space.shape
dummy_input = torch.randn(1, *observation_size)
torch.onnx.export(
    onnx_policy,
    dummy_input,
    f"{name}.onnx",
    opset_version=12,
    input_names=["input"],
)

In [8]:
import onnx
import copy

In [9]:
model = onnx.load(f"{name}.onnx")

In [10]:
model.graph.node.pop(0)
model.graph.node.pop(0)
model.graph.node.pop(-1)
model.graph.node.pop(-1)
model.graph.node.pop(-1)
model.graph.node

[input: "/policy/q_net/features_extractor/flatten/Flatten_output_0"
input: "policy.q_net.q_net.0.weight"
input: "policy.q_net.q_net.0.bias"
output: "/policy/q_net/q_net/q_net.0/Gemm_output_0"
name: "/policy/q_net/q_net/q_net.0/Gemm"
op_type: "Gemm"
attribute {
  name: "alpha"
  f: 1
  type: FLOAT
}
attribute {
  name: "beta"
  f: 1
  type: FLOAT
}
attribute {
  name: "transB"
  i: 1
  type: INT
}
, input: "/policy/q_net/q_net/q_net.0/Gemm_output_0"
output: "/policy/q_net/q_net/q_net.1/Relu_output_0"
name: "/policy/q_net/q_net/q_net.1/Relu"
op_type: "Relu"
, input: "/policy/q_net/q_net/q_net.1/Relu_output_0"
input: "policy.q_net.q_net.2.weight"
input: "policy.q_net.q_net.2.bias"
output: "/policy/q_net/q_net/q_net.2/Gemm_output_0"
name: "/policy/q_net/q_net/q_net.2/Gemm"
op_type: "Gemm"
attribute {
  name: "alpha"
  f: 1
  type: FLOAT
}
attribute {
  name: "beta"
  f: 1
  type: FLOAT
}
attribute {
  name: "transB"
  i: 1
  type: INT
}
, input: "/policy/q_net/q_net/q_net.2/Gemm_output_0"


In [11]:
model.graph.output[0].name = "/policy/q_net/q_net/q_net.4/Gemm_output_0"
model.graph.output[0].type.tensor_type.shape.dim[0].dim_value=3
model.graph.output[0].type.tensor_type.elem_type=1

In [12]:
dim_template = copy.deepcopy(model.graph.output[0].type.tensor_type.shape.dim[0])
model.graph.output[0].type.tensor_type.shape.dim.append(dim_template)
model.graph.output[0].type.tensor_type.shape.dim[0].dim_value=1

In [13]:
model.graph.output[0].type.tensor_type.shape.dim

[dim_value: 1
, dim_value: 3
]

In [14]:
model.graph.node[0].input[0] = "input"

In [15]:
model.graph.input[0].type.tensor_type.shape.dim.pop(-1)
#model.graph.input[0].type.tensor_type.shape.dim.pop(-1)
model.graph.input[0].type.tensor_type.shape.dim[1].dim_value = 25

In [16]:
onnx.save(model, f"{name}-adjusted.onnx")

In [17]:
onnx_model = onnx.load(f"{name}-adjusted.onnx")
onnx.checker.check_model(onnx_model)

In [18]:
import onnxruntime as ort
import numpy as np
ort_sess = ort.InferenceSession(f"{name}-adjusted.onnx")
x = np.random.normal(size=(1, 25)).astype(np.float32)
ort_sess.run(None, {'input': x})

[array([[3.7166219, 1.4289604, 6.3589163]], dtype=float32)]